Live video feed processing

In [ ]:
import cv2
import torch
import numpy as np

# Defining the initial secondary frame
prev_output = None  # To store the previous frame for optical flow comparison

# Downloading the MiDaS model
midas = torch.hub.load('intel-isl/MiDaS', 'MiDaS_small')
device = torch.device('cpu')
midas.to(device)
midas.eval()

transforms = torch.hub.load('intel-isl/MiDaS', 'transforms')
transform = transforms.small_transform

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Transforming the image
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    imgbatch = transform(img).to(device)

    with torch.no_grad():
        prediction = midas(imgbatch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode='bicubic',
            align_corners=False
        ).squeeze()

        output = prediction.cpu().numpy()

        # Normalize output
        output_normalized = cv2.normalize(output, None, 0, 255, cv2.NORM_MINMAX)
        output_normalized = np.uint8(output_normalized)

        # Apply Median filter to preserve edges and reduce noise
        output_filtered = cv2.medianBlur(output_normalized, 5)

    # If this is the first frame, skip comparison
    if prev_output is None:
        prev_output = output_filtered
        continue

    # Optical Flow for pixel-level movement detection (Farneback method)
    flow = cv2.calcOpticalFlowFarneback(prev_output, output_filtered, None, 
                                        0.5, 3, 15, 3, 5, 1.2, 0)
    mag, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    motion_map = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    motion_map = np.uint8(motion_map)

    # Segmenting into panes: left, middle, right
    height, width = motion_map.shape
    part_width = width // 3

    # Slice the motion map into three equal parts
    left = motion_map[:, :part_width]
    middle = motion_map[:, part_width:2 * part_width]
    right = motion_map[:, 2 * part_width:]

    # Calculating pixel-level motion in each pane
    left_motion = np.mean(left)
    middle_motion = np.mean(middle)
    right_motion = np.mean(right)

    # Dynamic threshold calculation based on standard deviation of motion
    motion_avg = np.mean([left_motion, middle_motion, right_motion])
    motion_std = np.std([left_motion, middle_motion, right_motion])
    dynamic_threshold = motion_avg + motion_std * 0.5  # Adjust factor as needed

    alert_messages = []

    # Classifying panes where significant motion is detected
    if left_motion > dynamic_threshold:
        alert_messages.append("Obstacle moving in the Left Pane")
        cv2.rectangle(frame, (0, 0), (part_width, height), (0, 0, 255), 2)
    if middle_motion > dynamic_threshold:
        alert_messages.append("Obstacle moving in the Middle Pane")
        cv2.rectangle(frame, (part_width, 0), (2 * part_width, height), (0, 0, 255), 2)
    if right_motion > dynamic_threshold:
        alert_messages.append("Obstacle moving in the Right Pane")
        cv2.rectangle(frame, (2 * part_width, 0), (width, height), (0, 0, 255), 2)

    # Display the alert messages for panes where obstacles are moving
    for message in alert_messages:
        print(message)

    # Update the previous frame for optical flow calculation in the next iteration
    prev_output = output_filtered

    # Display the output depth and motion maps alongside the frame with highlighted panes
    cv2.imshow('Depth Prediction (Filtered)', output_filtered)
    cv2.imshow('Motion Map', motion_map)
    cv2.imshow('CV2Frame', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
print(prev_output)

Local file processing

In [1]:
import cv2
import torch
import numpy as np

# Defining the initial secondary frame
prev_output = None  # To store the previous frame for optical flow comparison

# Downloading the MiDaS model
midas = torch.hub.load('intel-isl/MiDaS', 'MiDaS_small')
device = torch.device('cpu')
midas.to(device)
midas.eval()

transforms = torch.hub.load('intel-isl/MiDaS', 'transforms')
transform = transforms.small_transform

# Path to input video file (replace with your video path)
video_path = '/Users/adil/Desktop/Codes/Image Captioning/optical_flow/test_clip.MOV'
cap = cv2.VideoCapture(video_path)

# Define video writer to save the processed output
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_path = 'processed_output.mp4'  # Path to save processed video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

depth_output_path = 'depth_output_no_ma.mp4'
depth_out = cv2.VideoWriter(depth_output_path, fourcc, fps, (frame_width, frame_height), False)

# Dictionary to store frame count and detected pane
frame_obstacle_dict = {}
frame_count = 0
frames = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)
    

    # Transforming the image
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    imgbatch = transform(img).to(device)

    with torch.no_grad():
        prediction = midas(imgbatch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode='bicubic',
            align_corners=False
        ).squeeze()

        output = prediction.cpu().numpy()

        # Normalize output
        output_normalized = cv2.normalize(output, None, 0, 255, cv2.NORM_MINMAX)
        output_normalized = np.uint8(output_normalized)

        # Apply Median filter to preserve edges and reduce noise
        output_filtered = cv2.medianBlur(output_normalized, 5)
        # Save the averaged depth frame to the output video
        depth_out.write(output_filtered)

    # If this is the first frame, skip comparison
    if prev_output is None:
        prev_output = output_filtered
        continue

    # Optical Flow for pixel-level movement detection (Farneback method)
    flow = cv2.calcOpticalFlowFarneback(prev_output, output_filtered, None, 
                                        0.5, 3, 15, 3, 5, 1.2, 0)
    mag, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
    motion_map = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    motion_map = np.uint8(motion_map)

    # Segmenting into panes: left, middle, right
    height, width = motion_map.shape
    part_width = width // 3

    # Slice the motion map into three equal parts
    left = motion_map[:, :part_width]
    middle = motion_map[:, part_width:2 * part_width]
    right = motion_map[:, 2 * part_width:]

    # Calculating pixel-level motion in each pane
    left_motion = np.mean(left)
    middle_motion = np.mean(middle)
    right_motion = np.mean(right)

    # Dynamic threshold calculation based on standard deviation of motion
    motion_avg = np.mean([left_motion, middle_motion, right_motion])
    motion_std = np.std([left_motion, middle_motion, right_motion])
    dynamic_threshold = motion_avg + motion_std * 0.5  # Adjust factor as needed

    alert_messages = []
    detected_pane = None  # Variable to store detected pane

    # Classifying panes where significant motion is detected
    if left_motion > dynamic_threshold:
        alert_messages.append("Obstacle moving in the Left Pane")
        cv2.rectangle(frame, (0, 0), (part_width, height), (0, 0, 255), 2)
        detected_pane = "Left"
    if middle_motion > dynamic_threshold:
        alert_messages.append("Obstacle moving in the Middle Pane")
        cv2.rectangle(frame, (part_width, 0), (2 * part_width, height), (0, 0, 255), 2)
        detected_pane = "Middle"
    if right_motion > dynamic_threshold:
        alert_messages.append("Obstacle moving in the Right Pane")
        cv2.rectangle(frame, (2 * part_width, 0), (width, height), (0, 0, 255), 2)
        detected_pane = "Right"

    # Display the alert messages for panes where obstacles are moving
    for message in alert_messages:
        print(f"Frame {frame_count}: {message}")

    # Store detected pane in dictionary
    if detected_pane:
        frame_obstacle_dict[frame_count] = detected_pane
        frame_count += 1
    # Update the previous frame for optical flow calculation in the next iteration
    prev_output = output_filtered

    # Save the processed frame to output video
    out.write(frame)

    # Display the output depth and motion maps alongside the frame with highlighted panes
    # cv2.imshow('Depth Prediction (Filtered)', output_filtered)
    # cv2.imshow('Motion Map', motion_map)
    # cv2.imshow('CV2Frame', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()  # Save the output video
cv2.destroyAllWindows()

Using cache found in /Users/adil/.cache/torch/hub/intel-isl_MiDaS_master
/Users/adil/Desktop/Codes/Image Captioning/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading weights:  None


Using cache found in /Users/adil/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Using cache found in /Users/adil/.cache/torch/hub/intel-isl_MiDaS_master


Frame 0: Obstacle moving in the Left Pane
Frame 1: Obstacle moving in the Left Pane
Frame 2: Obstacle moving in the Left Pane
Frame 3: Obstacle moving in the Left Pane
Frame 4: Obstacle moving in the Left Pane
Frame 5: Obstacle moving in the Left Pane
Frame 6: Obstacle moving in the Left Pane
Frame 7: Obstacle moving in the Left Pane
Frame 8: Obstacle moving in the Left Pane
Frame 9: Obstacle moving in the Left Pane
Frame 10: Obstacle moving in the Left Pane
Frame 11: Obstacle moving in the Left Pane
Frame 12: Obstacle moving in the Left Pane
Frame 13: Obstacle moving in the Left Pane
Frame 14: Obstacle moving in the Left Pane
Frame 15: Obstacle moving in the Left Pane
Frame 16: Obstacle moving in the Left Pane
Frame 17: Obstacle moving in the Left Pane
Frame 18: Obstacle moving in the Left Pane
Frame 19: Obstacle moving in the Left Pane
Frame 20: Obstacle moving in the Left Pane
Frame 21: Obstacle moving in the Left Pane
Frame 22: Obstacle moving in the Left Pane
Frame 23: Obstacle mo

In [2]:
import json

# Save the dictionary to a JSON file
with open('frame_obstacle_dict.json', 'w') as json_file:
    json.dump(frame_obstacle_dict, json_file)

In [2]:
depth_out.release() 

In [3]:
frames[0]

array([[[137, 136, 138],
        [137, 136, 138],
        [138, 137, 139],
        ...,
        [187, 183, 188],
        [187, 183, 188],
        [187, 183, 188]],

       [[137, 136, 138],
        [137, 136, 138],
        [138, 137, 139],
        ...,
        [187, 183, 188],
        [187, 183, 188],
        [187, 183, 188]],

       [[137, 136, 138],
        [137, 136, 138],
        [138, 137, 139],
        ...,
        [187, 183, 188],
        [187, 183, 188],
        [187, 183, 188]],

       ...,

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]]

In [ ]:
import os
output_folder = 'saved_frames'  # Directory where the frames will be saved

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Iterate through the list of frames and save each one
for idx, frame in enumerate(frames):
    frame_filename = os.path.join(output_folder, f'frame_{idx}.jpg')  # Save as JPG files
    cv2.imwrite(frame_filename, frame)

optical flow + moving average

In [1]:
import cv2
import numpy as np

# Defining the initial secondary frame
prev_output = None  # To store the previous frame for optical flow comparison
moving_average_frames = 5  # Number of frames for moving average
frame_buffer = []  # Buffer to store frames for temporal smoothing

# Path to input video file (replace with your video path)
video_path = '/Users/adil/Desktop/Codes/Image Captioning/optical_flow/test_clip.MOV'
cap = cv2.VideoCapture(video_path)

# Define video writer to save the processed output
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_path = 'five_ma_woDepth_processed_output.mp4'  # Path to save processed video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

flow_output_path = 'five_flow_woDepth_output.mp4'
flow_output = cv2.VideoWriter(flow_output_path, fourcc, fps, (frame_width, frame_height), False)

# Dictionary to store frame count and detected pane
frame_obstacle_dict = {}
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Convert the frame to grayscale for optical flow calculation
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Add the current frame to the buffer for temporal smoothing
    frame_buffer.append(gray_frame)

    # If fewer than `moving_average_frames` are in the buffer, skip to next frame
    if len(frame_buffer) < moving_average_frames:
        prev_output = gray_frame
        continue

    # Compute the moving average of the frames
    averaged_frame = np.mean(frame_buffer[-moving_average_frames:], axis=0)
    averaged_frame = np.uint8(averaged_frame)

    # Optical Flow for pixel-level movement detection (Farneback method)
    if prev_output is not None:
        flow = cv2.calcOpticalFlowFarneback(prev_output, averaged_frame, None, 
                                            0.5, 3, 15, 3, 5, 1.2, 0)
        mag, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        motion_map = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        motion_map = np.uint8(motion_map)
        flow_output.write(motion_map)

        # Segmenting into panes: left, middle, right
        height, width = motion_map.shape
        part_width = width // 3

        # Slice the motion map into three equal parts
        left = motion_map[:, :part_width]
        middle = motion_map[:, part_width:2 * part_width]
        right = motion_map[:, 2 * part_width:]

        # Calculating pixel-level motion in each pane
        left_motion = np.mean(left)
        middle_motion = np.mean(middle)
        right_motion = np.mean(right)

        # Dynamic threshold calculation based on standard deviation of motion
        motion_avg = np.mean([left_motion, middle_motion, right_motion])
        motion_std = np.std([left_motion, middle_motion, right_motion])
        dynamic_threshold = motion_avg + motion_std * 0.5  # Adjust factor as needed

        alert_messages = []
        detected_pane = None  # Variable to store detected pane

        # Classifying panes where significant motion is detected
        if left_motion > dynamic_threshold:
            alert_messages.append("Obstacle moving in the Left Pane")
            cv2.rectangle(frame, (0, 0), (part_width, height), (0, 0, 255), 2)
            detected_pane = "Left"
        if middle_motion > dynamic_threshold:
            alert_messages.append("Obstacle moving in the Middle Pane")
            cv2.rectangle(frame, (part_width, 0), (2 * part_width, height), (0, 0, 255), 2)
            detected_pane = "Middle"
        if right_motion > dynamic_threshold:
            alert_messages.append("Obstacle moving in the Right Pane")
            cv2.rectangle(frame, (2 * part_width, 0), (width, height), (0, 0, 255), 2)
            detected_pane = "Right"

        # Display the alert messages for panes where obstacles are moving
        for message in alert_messages:
            print(f"Frame {frame_count}: {message}")

        # Store detected pane in dictionary
        if detected_pane:
            frame_obstacle_dict[frame_count] = detected_pane

    # Update the previous frame for optical flow calculation in the next iteration
    prev_output = averaged_frame
    frame_count += 1

    # Save the processed frame to output video
    out.write(frame)

    # Display the motion map and the original frame
    # cv2.imshow('Motion Map', motion_map)
    # cv2.imshow('Original Frame', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()  # Save the output video
flow_output.release()
cv2.destroyAllWindows()


Frame 0: Obstacle moving in the Right Pane
Frame 1: Obstacle moving in the Right Pane
Frame 2: Obstacle moving in the Right Pane
Frame 3: Obstacle moving in the Middle Pane
Frame 3: Obstacle moving in the Right Pane
Frame 4: Obstacle moving in the Right Pane
Frame 5: Obstacle moving in the Right Pane
Frame 6: Obstacle moving in the Middle Pane
Frame 6: Obstacle moving in the Right Pane
Frame 7: Obstacle moving in the Middle Pane
Frame 7: Obstacle moving in the Right Pane
Frame 8: Obstacle moving in the Middle Pane
Frame 8: Obstacle moving in the Right Pane
Frame 9: Obstacle moving in the Middle Pane
Frame 9: Obstacle moving in the Right Pane
Frame 10: Obstacle moving in the Middle Pane
Frame 10: Obstacle moving in the Right Pane
Frame 11: Obstacle moving in the Middle Pane
Frame 11: Obstacle moving in the Right Pane
Frame 12: Obstacle moving in the Middle Pane
Frame 12: Obstacle moving in the Right Pane
Frame 13: Obstacle moving in the Middle Pane
Frame 13: Obstacle moving in the Right

plain optical flow output

In [ ]:
import cv2
import numpy as np

# Path to input video file (replace with your video path)
video_path = '/Users/adil/Desktop/Codes/Image Captioning/optical_flow/test_clip.MOV'
cap = cv2.VideoCapture(video_path)

# Define video writer to save the optical flow output
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_path = 'plain_optical_flow_output.mp4'  # Path to save processed video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height), False)

# Variable to store the previous frame
prev_gray = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Initialize the previous frame if it's the first frame
    if prev_gray is None:
        prev_gray = gray_frame
        continue

    # Calculate Optical Flow using Farneback method
    flow = cv2.calcOpticalFlowFarneback(prev_gray, gray_frame, None, 
                                        0.5, 3, 15, 3, 5, 1.2, 0)

    # Compute magnitude and angle of flow
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # Normalize the magnitude to the range 0-255 for visualization
    motion_map = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
    motion_map = np.uint8(motion_map)

    # Save the optical flow (motion map) to the video
    out.write(motion_map)

    # Display the optical flow map
    # cv2.imshow('Optical Flow', motion_map)

    # Update the previous frame for the next iteration
    prev_gray = gray_frame

    # Break the loop if 'q' is pressed
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


MA approach

In [2]:
import cv2
import torch
import numpy as np

# Defining the initial secondary frame
prev_output = None  # To store the previous frame for optical flow comparison
moving_average_frames = 5  # Number of frames for moving average
depth_buffer = []  # Buffer to store frames for temporal smoothing

# Downloading the MiDaS model
midas = torch.hub.load('intel-isl/MiDaS', 'MiDaS_small')
device = torch.device('cpu')
midas.to(device)
midas.eval()

transforms = torch.hub.load('intel-isl/MiDaS', 'transforms')
transform = transforms.small_transform

# Path to input video file (replace with your video path)
video_path = '/Users/adil/Desktop/Codes/Image Captioning/optical_flow/test_clip.MOV'
cap = cv2.VideoCapture(video_path)

# Define video writer to save the processed output
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_path = 'five_ma_processed_output.mp4'  # Path to save processed video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Video writer for saving the smoothed depth map
depth_output_path = 'five_depth_output.mp4'
depth_out = cv2.VideoWriter(depth_output_path, fourcc, fps, (frame_width, frame_height), False)

flow_output_path = 'five_flow_output.mp4'
flow_output = cv2.VideoWriter(flow_output_path, fourcc, fps, (frame_width, frame_height), False)


# Dictionary to store frame count and detected pane
frame_obstacle_dict = {}
frame_count = 0
frames = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)

    # Transforming the image
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    imgbatch = transform(img).to(device)

    with torch.no_grad():
        prediction = midas(imgbatch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode='bicubic',
            align_corners=False
        ).squeeze()

        output = prediction.cpu().numpy()

        # Normalize output
        output_normalized = cv2.normalize(output, None, 0, 255, cv2.NORM_MINMAX)
        output_normalized = np.uint8(output_normalized)

        # Apply Median filter to preserve edges and reduce noise
        output_filtered = cv2.medianBlur(output_normalized, 5)

        # Add the depth map to the buffer for temporal smoothing
        depth_buffer.append(output_filtered)

        # If fewer than `moving_average_frames` are in the buffer, continue to next frame
        if len(depth_buffer) < moving_average_frames:
            prev_output = output_filtered
            continue

        # Compute the moving average of depth maps
        averaged_depth = np.mean(depth_buffer[-moving_average_frames:], axis=0)
        averaged_depth = np.uint8(averaged_depth)

        # Save the averaged depth frame to the output video
        depth_out.write(averaged_depth)

        # Optical Flow for pixel-level movement detection (Farneback method)
        if prev_output is not None:
            flow = cv2.calcOpticalFlowFarneback(prev_output, averaged_depth, None, 
                                                0.5, 3, 15, 3, 5, 1.2, 0)
            mag, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
            motion_map = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
            motion_map = np.uint8(motion_map)
            flow_output.write(motion_map)

            # Segmenting into panes: left, middle, right
            height, width = motion_map.shape
            part_width = width // 3

            # Slice the motion map into three equal parts
            left = motion_map[:, :part_width]
            middle = motion_map[:, part_width:2 * part_width]
            right = motion_map[:, 2 * part_width:]

            # Calculating pixel-level motion in each pane
            left_motion = np.mean(left)
            middle_motion = np.mean(middle)
            right_motion = np.mean(right)

            # Dynamic threshold calculation based on standard deviation of motion
            motion_avg = np.mean([left_motion, middle_motion, right_motion])
            motion_std = np.std([left_motion, middle_motion, right_motion])
            dynamic_threshold = motion_avg + motion_std * 0.5  # Adjust factor as needed

            alert_messages = []
            detected_pane = None  # Variable to store detected pane

            # Classifying panes where significant motion is detected
            if left_motion > dynamic_threshold:
                alert_messages.append("Obstacle moving in the Left Pane")
                cv2.rectangle(frame, (0, 0), (part_width, height), (0, 0, 255), 2)
                detected_pane = "Left"
            if middle_motion > dynamic_threshold:
                alert_messages.append("Obstacle moving in the Middle Pane")
                cv2.rectangle(frame, (part_width, 0), (2 * part_width, height), (0, 0, 255), 2)
                detected_pane = "Middle"
            if right_motion > dynamic_threshold:
                alert_messages.append("Obstacle moving in the Right Pane")
                cv2.rectangle(frame, (2 * part_width, 0), (width, height), (0, 0, 255), 2)
                detected_pane = "Right"

            # Display the alert messages for panes where obstacles are moving
            for message in alert_messages:
                print(f"Frame {frame_count}: {message}")

            # Store detected pane in dictionary
            if detected_pane:
                frame_obstacle_dict[frame_count] = detected_pane

        # Update the previous frame for optical flow calculation in the next iteration
        prev_output = averaged_depth
        frame_count += 1

    # Save the processed frame to output video
    out.write(frame)

    # Display the depth map and the motion map
    # cv2.imshow('Averaged Depth Map', averaged_depth)
    # cv2.imshow('Motion Map', motion_map)
    # cv2.imshow('CV2Frame', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()  # Save the output video
depth_out.release()  # Save the depth map video
flow_output.release()
cv2.destroyAllWindows()


Using cache found in /Users/adil/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /Users/adil/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Using cache found in /Users/adil/.cache/torch/hub/intel-isl_MiDaS_master


Frame 0: Obstacle moving in the Left Pane
Frame 1: Obstacle moving in the Left Pane
Frame 2: Obstacle moving in the Left Pane
Frame 3: Obstacle moving in the Left Pane
Frame 4: Obstacle moving in the Left Pane
Frame 5: Obstacle moving in the Left Pane
Frame 6: Obstacle moving in the Left Pane
Frame 7: Obstacle moving in the Left Pane
Frame 8: Obstacle moving in the Left Pane
Frame 9: Obstacle moving in the Left Pane
Frame 10: Obstacle moving in the Left Pane
Frame 11: Obstacle moving in the Left Pane
Frame 12: Obstacle moving in the Left Pane
Frame 13: Obstacle moving in the Left Pane
Frame 14: Obstacle moving in the Left Pane
Frame 15: Obstacle moving in the Left Pane
Frame 16: Obstacle moving in the Left Pane
Frame 17: Obstacle moving in the Left Pane
Frame 18: Obstacle moving in the Left Pane
Frame 19: Obstacle moving in the Left Pane
Frame 20: Obstacle moving in the Left Pane
Frame 21: Obstacle moving in the Left Pane
Frame 22: Obstacle moving in the Left Pane
Frame 23: Obstacle mo

claude approach

In [4]:
import cv2
import torch
import numpy as np

# Defining the initial secondary frame
prev_output = None  # To store the previous frame for optical flow comparison
moving_average_frames = 5  # Number of frames for moving average
depth_buffer = []  # Buffer to store frames for temporal smoothing

# Downloading the MiDaS model
midas = torch.hub.load('intel-isl/MiDaS', 'MiDaS_small')
device = torch.device('cpu')
midas.to(device)
midas.eval()

transforms = torch.hub.load('intel-isl/MiDaS', 'transforms')
transform = transforms.small_transform

# Path to input video file (replace with your video path)
video_path = '/Users/adil/Desktop/Codes/Image Captioning/optical_flow/test_clip.MOV'
cap = cv2.VideoCapture(video_path)

# Define video writer to save the processed output
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_path = 'claude_processed_output.mp4'  # Path to save processed video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Video writer for saving the smoothed depth map
depth_output_path = 'c_depth_output.mp4'
depth_out = cv2.VideoWriter(depth_output_path, fourcc, fps, (frame_width, frame_height), False)

flow_output_path = 'c_flow_output.mp4'
flow_output = cv2.VideoWriter(flow_output_path, fourcc, fps, (frame_width, frame_height), False)


# Dictionary to store frame count and detected pane
frame_obstacle_dict = {}
frame_count = 0
frames = []
# New function for depth-based obstacle detection
def detect_obstacles_depth(depth_map, min_depth, max_depth):
    obstacle_mask = cv2.inRange(depth_map, min_depth, max_depth)
    return obstacle_mask


def detect_obstacles_relative_depth(depth_map, threshold_percentile=10):
    # Normalize the depth map to 0-255 range
    depth_norm = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    
    # Calculate the threshold value based on the percentile of depth values
    threshold = np.percentile(depth_norm, threshold_percentile)
    
    # Create a binary mask of obstacles (closer objects are brighter in the depth map)
    obstacle_mask = (depth_norm > threshold).astype(np.uint8) * 255
    
    return obstacle_mask

def detect_edges(depth_map):
    # Normalize the depth map to 0-255 range
    depth_norm = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(depth_norm, (5, 5), 0)
    
    # Use Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)
    
    return edges

# Modified main loop
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)

    # ... (MiDaS depth estimation code remains the same)
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    imgbatch = transform(img).to(device)
    with torch.no_grad():
        prediction = midas(imgbatch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode='bicubic',
            align_corners=False
        ).squeeze()

        output = prediction.cpu().numpy()

        # Normalize output
        output_normalized = cv2.normalize(output, None, 0, 255, cv2.NORM_MINMAX)
        output_normalized = np.uint8(output_normalized)

        # Apply Median filter to preserve edges and reduce noise
        output_filtered = cv2.medianBlur(output_normalized, 5)

        # Add the depth map to the buffer for temporal smoothing
        depth_buffer.append(output_filtered)

        # If fewer than `moving_average_frames` are in the buffer, continue to next frame
        if len(depth_buffer) < moving_average_frames:
            prev_output = output_filtered
            continue

        # Compute the moving average of depth maps
        # averaged_depth = np.mean(depth_buffer[-moving_average_frames:], axis=0)
        # averaged_depth = np.uint8(averaged_depth)

        averaged_depth = np.mean(depth_buffer[-moving_average_frames:], axis=0)

        # Relative depth-based obstacle detection
        obstacle_mask = detect_obstacles_relative_depth(averaged_depth, threshold_percentile=10)

        # Edge detection
        edges = detect_edges(averaged_depth)

        # Combine obstacle mask and edges
        combined_mask = cv2.bitwise_or(obstacle_mask, edges)

        # Optical Flow (keep this for dynamic obstacle detection)
        if prev_output is not None:
            flow = cv2.calcOpticalFlowFarneback(prev_output, averaged_depth, None, 
                                                0.5, 3, 15, 3, 5, 1.2, 0)
            mag, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
            motion_map = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

            # Combine motion map with depth-based detection
            final_mask = cv2.bitwise_or(combined_mask, motion_map)
        else:
            final_mask = combined_mask

        # Segment the final mask into panes
        height, width = final_mask.shape
        part_width = width // 3

        left = final_mask[:, :part_width]
        middle = final_mask[:, part_width:2 * part_width]
        right = final_mask[:, 2 * part_width:]

        # Calculate the percentage of obstacle pixels in each pane
        left_obstacle = np.mean(left) / 255 * 100
        middle_obstacle = np.mean(middle) / 255 * 100
        right_obstacle = np.mean(right) / 255 * 100

        # Dynamic threshold calculation
        obstacle_avg = np.mean([left_obstacle, middle_obstacle, right_obstacle])
        obstacle_std = np.std([left_obstacle, middle_obstacle, right_obstacle])
        dynamic_threshold = obstacle_avg + obstacle_std * 0.5  # Adjust factor as needed

        # Classify panes with significant obstacles
        alert_messages = []
        if left_obstacle > dynamic_threshold:
            alert_messages.append("Obstacle detected in the Left Pane")
            cv2.rectangle(frame, (0, 0), (part_width, height), (0, 0, 255), 2)
        if middle_obstacle > dynamic_threshold:
            alert_messages.append("Obstacle detected in the Middle Pane")
            cv2.rectangle(frame, (part_width, 0), (2 * part_width, height), (0, 0, 255), 2)
        if right_obstacle > dynamic_threshold:
            alert_messages.append("Obstacle detected in the Right Pane")
            cv2.rectangle(frame, (2 * part_width, 0), (width, height), (0, 0, 255), 2)

        # Display alert messages
        for message in alert_messages:
            print(f"Frame {frame_count}: {message}")

        # Update previous frame for next iteration
        prev_output = averaged_depth
        frame_count += 1

    # Save processed frame and masks
    out.write(frame)
    depth_out.write(cv2.normalize(averaged_depth, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U))
    # cv2.imwrite(f'obstacle_mask_{frame_count}.jpg', final_mask)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
depth_out.release()
cv2.destroyAllWindows()

Using cache found in /Users/adil/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /Users/adil/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Using cache found in /Users/adil/.cache/torch/hub/intel-isl_MiDaS_master


Frame 0: Obstacle detected in the Left Pane
Frame 0: Obstacle detected in the Right Pane
Frame 1: Obstacle detected in the Left Pane
Frame 1: Obstacle detected in the Right Pane
Frame 2: Obstacle detected in the Left Pane
Frame 2: Obstacle detected in the Right Pane
Frame 3: Obstacle detected in the Left Pane
Frame 3: Obstacle detected in the Right Pane
Frame 4: Obstacle detected in the Left Pane
Frame 4: Obstacle detected in the Right Pane
Frame 5: Obstacle detected in the Left Pane
Frame 5: Obstacle detected in the Right Pane
Frame 6: Obstacle detected in the Left Pane
Frame 6: Obstacle detected in the Right Pane
Frame 7: Obstacle detected in the Left Pane
Frame 7: Obstacle detected in the Right Pane
Frame 8: Obstacle detected in the Left Pane
Frame 8: Obstacle detected in the Right Pane
Frame 9: Obstacle detected in the Left Pane
Frame 9: Obstacle detected in the Right Pane
Frame 10: Obstacle detected in the Left Pane
Frame 10: Obstacle detected in the Right Pane
Frame 11: Obstacle 

In [3]:
out.release()
depth_out.release()

Wall problem solved

In [1]:
import cv2
import torch
import numpy as np

# Defining the initial secondary frame
prev_output = None  # To store the previous frame for optical flow comparison
moving_average_frames = 5  # Number of frames for moving average
depth_buffer = []  # Buffer to store frames for temporal smoothing

# Downloading the MiDaS model
midas = torch.hub.load('intel-isl/MiDaS', 'MiDaS_small')
device = torch.device('cpu')
midas.to(device)
midas.eval()

transforms = torch.hub.load('intel-isl/MiDaS', 'transforms')
transform = transforms.small_transform

# Path to input video file (replace with your video path)
video_path = '/Users/adil/Desktop/Codes/Image Captioning/optical_flow/Screen Recording 2024-10-14 at 3.10.52 PM.mov'
cap = cv2.VideoCapture(video_path)

# Define video writer to save the processed output
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_path = 'five_ma_processed_output.mp4'  # Path to save processed video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Video writer for saving the smoothed depth map
depth_output_path = 'five_depth_output.mp4'
depth_out = cv2.VideoWriter(depth_output_path, fourcc, fps, (frame_width, frame_height), False)

flow_output_path = 'five_flow_output.mp4'
flow_output = cv2.VideoWriter(flow_output_path, fourcc, fps, (frame_width, frame_height), False)

# Dictionary to store frame count and detected pane
frame_obstacle_dict = {}
frame_count = 0
frames = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)

    # Transforming the image
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    imgbatch = transform(img).to(device)

    with torch.no_grad():
        prediction = midas(imgbatch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode='bicubic',
            align_corners=False
        ).squeeze()

        output = prediction.cpu().numpy()

        # Normalize output
        output_normalized = cv2.normalize(output, None, 0, 255, cv2.NORM_MINMAX)
        output_normalized = np.uint8(output_normalized)

        # Apply Median filter to preserve edges and reduce noise
        output_filtered = cv2.medianBlur(output_normalized, 5)

        # Add the depth map to the buffer for temporal smoothing
        depth_buffer.append(output_filtered)

        # If fewer than `moving_average_frames` are in the buffer, continue to next frame
        if len(depth_buffer) < moving_average_frames:
            prev_output = output_filtered
            continue

        # Compute the moving average of depth maps
        averaged_depth = np.mean(depth_buffer[-moving_average_frames:], axis=0)
        averaged_depth = np.uint8(averaged_depth)

        # Save the averaged depth frame to the output video
        depth_out.write(averaged_depth)

        # Optical Flow for pixel-level movement detection (Farneback method)
        if prev_output is not None:
            flow = cv2.calcOpticalFlowFarneback(prev_output, averaged_depth, None, 
                                                0.5, 3, 15, 3, 5, 1.2, 0)
            mag, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
            motion_map = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
            motion_map = np.uint8(motion_map)
            flow_output.write(motion_map)

            # Segmenting into panes: left, middle, right (for both depth and motion maps)
            height, width = motion_map.shape
            part_width = width // 3

            # Slicing the depth map into three equal parts
            left_depth = averaged_depth[:, :part_width]
            middle_depth = averaged_depth[:, part_width:2 * part_width]
            right_depth = averaged_depth[:, 2 * part_width:]

            # Thresholds for detecting if a pane is "covered" in the depth map
            depth_coverage_threshold = 150  # Define a depth threshold for coverage
            coverage_percentage_threshold = 80  # If more than 80% of the pane is covered

            # Check for depth coverage in each pane
            left_depth_coverage = np.mean(left_depth > depth_coverage_threshold) * 100
            middle_depth_coverage = np.mean(middle_depth > depth_coverage_threshold) * 100
            right_depth_coverage = np.mean(right_depth > depth_coverage_threshold) * 100

            alert_messages = []
            detected_pane = None  # Variable to store detected pane

            # Check if the left pane is fully covered based on depth map
            if left_depth_coverage > coverage_percentage_threshold:
                alert_messages.append("Left Pane Fully Covered by Obstacle (Depth Map)")
                cv2.rectangle(frame, (0, 0), (part_width, height), (255, 0, 0), 2)  # Blue box for depth
                detected_pane = "Left"

            # Check if the middle pane is fully covered based on depth map
            if middle_depth_coverage > coverage_percentage_threshold:
                alert_messages.append("Middle Pane Fully Covered by Obstacle (Depth Map)")
                cv2.rectangle(frame, (part_width, 0), (2 * part_width, height), (255, 0, 0), 2)  # Blue box for depth
                detected_pane = "Middle"

            # Check if the right pane is fully covered based on depth map
            if right_depth_coverage > coverage_percentage_threshold:
                alert_messages.append("Right Pane Fully Covered by Obstacle (Depth Map)")
                cv2.rectangle(frame, (2 * part_width, 0), (width, height), (255, 0, 0), 2)  # Blue box for depth
                detected_pane = "Right"

            # Segmenting the motion map into left, middle, and right
            left_motion = motion_map[:, :part_width]
            middle_motion = motion_map[:, part_width:2 * part_width]
            right_motion = motion_map[:, 2 * part_width:]

            # Calculating pixel-level motion in each pane
            left_motion_intensity = np.mean(left_motion)
            middle_motion_intensity = np.mean(middle_motion)
            right_motion_intensity = np.mean(right_motion)

            # Dynamic threshold calculation based on standard deviation of motion
            motion_avg = np.mean([left_motion_intensity, middle_motion_intensity, right_motion_intensity])
            motion_std = np.std([left_motion_intensity, middle_motion_intensity, right_motion_intensity])
            dynamic_threshold = motion_avg + motion_std * 0.5  # Adjust factor as needed

            # Classifying panes where significant motion is detected
            if left_motion_intensity > dynamic_threshold:
                alert_messages.append("Obstacle moving in the Left Pane")
                cv2.rectangle(frame, (0, 0), (part_width, height), (0, 0, 255), 2)  # Red box for motion
                detected_pane = "Left"

            if middle_motion_intensity > dynamic_threshold:
                alert_messages.append("Obstacle moving in the Middle Pane")
                cv2.rectangle(frame, (part_width, 0), (2 * part_width, height), (0, 0, 255), 2)  # Red box for motion
                detected_pane = "Middle"

            if right_motion_intensity > dynamic_threshold:
                alert_messages.append("Obstacle moving in the Right Pane")
                cv2.rectangle(frame, (2 * part_width, 0), (width, height), (0, 0, 255), 2)  # Red box for motion
                detected_pane = "Right"

            # Display the alert messages for panes where obstacles are moving or covered
            for message in alert_messages:
                print(f"Frame {frame_count}: {message}")

            # Store detected pane in dictionary
            if detected_pane:
                frame_obstacle_dict[frame_count] = detected_pane

        # Update the previous frame for optical flow calculation in the next iteration
        prev_output = averaged_depth
        frame_count += 1

    # Save the processed frame to output video
    out.write(frame)

    # Display the depth map and the motion map
    # cv2.imshow('Averaged Depth Map', averaged_depth)
    # cv2.imshow('Motion Map', motion_map)
    # cv2.imshow('CV2Frame', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()  # Save the output video
depth_out.release()  # Save the depth map video
flow_output.release()
cv2.destroyAllWindows()


Using cache found in /Users/adil/.cache/torch/hub/intel-isl_MiDaS_master
/Users/adil/Desktop/Codes/Image Captioning/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading weights:  None


Using cache found in /Users/adil/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master
Using cache found in /Users/adil/.cache/torch/hub/intel-isl_MiDaS_master


Frame 0: Obstacle moving in the Right Pane
Frame 1: Obstacle moving in the Right Pane
Frame 2: Obstacle moving in the Right Pane
Frame 3: Obstacle moving in the Right Pane
Frame 4: Obstacle moving in the Right Pane
Frame 5: Obstacle moving in the Right Pane
Frame 6: Obstacle moving in the Right Pane
Frame 7: Obstacle moving in the Right Pane
Frame 8: Obstacle moving in the Right Pane
Frame 9: Obstacle moving in the Right Pane
Frame 10: Obstacle moving in the Right Pane
Frame 11: Obstacle moving in the Right Pane
Frame 12: Obstacle moving in the Right Pane
Frame 13: Obstacle moving in the Right Pane
Frame 14: Obstacle moving in the Right Pane
Frame 15: Obstacle moving in the Right Pane
Frame 16: Obstacle moving in the Middle Pane
Frame 17: Obstacle moving in the Right Pane
Frame 18: Obstacle moving in the Left Pane
Frame 19: Obstacle moving in the Right Pane
Frame 20: Obstacle moving in the Right Pane
Frame 21: Obstacle moving in the Right Pane
Frame 22: Obstacle moving in the Right Pan

Final Approach

In [ ]:
import cv2
import torch
import numpy as np

# Defining the initial secondary frame
prev_output = None  # To store the previous frame for optical flow comparison
moving_average_frames = 5  # Number of frames for moving average
depth_buffer = []  # Buffer to store frames for temporal smoothing

# Downloading the MiDaS model
midas = torch.hub.load('intel-isl/MiDaS', 'MiDaS_small')
device = torch.device('cpu')
midas.to(device)
midas.eval()

transforms = torch.hub.load('intel-isl/MiDaS', 'transforms')
transform = transforms.small_transform

# Path to input video file (replace with your video path)
video_path = 'WhatsApp Video 2024-10-05 at 21.43.31_8e4501d7.mp4'
cap = cv2.VideoCapture(video_path)

# Define video writer to save the processed output
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
output_path = 'five_ma_processed_output.mp4'  # Path to save processed video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Video writer for saving the smoothed depth map
depth_output_path = 'five_depth_output.mp4'
depth_out = cv2.VideoWriter(depth_output_path, fourcc, fps, (frame_width, frame_height), False)

flow_output_path = 'five_flow_output.mp4'
flow_output = cv2.VideoWriter(flow_output_path, fourcc, fps, (frame_width, frame_height), False)

# Dictionary to store frame count and detected pane
frame_obstacle_dict = {}
frame_count = 0
frames = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)

    # Transforming the image
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    imgbatch = transform(img).to(device)

    with torch.no_grad():
        prediction = midas(imgbatch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode='bicubic',
            align_corners=False
        ).squeeze()

        output = prediction.cpu().numpy()

        # Normalize output
        output_normalized = cv2.normalize(output, None, 0, 255, cv2.NORM_MINMAX)
        output_normalized = np.uint8(output_normalized)

        # Apply Median filter to preserve edges and reduce noise
        output_filtered = cv2.medianBlur(output_normalized, 5)

        # Add the depth map to the buffer for temporal smoothing
        depth_buffer.append(output_filtered)

        # If fewer than `moving_average_frames` are in the buffer, continue to next frame
        if len(depth_buffer) < moving_average_frames:
            prev_output = output_filtered
            continue

        # Compute the moving average of depth maps
        averaged_depth = np.mean(depth_buffer[-moving_average_frames:], axis=0)
        averaged_depth = np.uint8(averaged_depth)

        # Save the averaged depth frame to the output video
        depth_out.write(averaged_depth)

        # Optical Flow for pixel-level movement detection (Farneback method)
        if prev_output is not None:
            flow = cv2.calcOpticalFlowFarneback(prev_output, averaged_depth, None, 
                                                0.5, 3, 15, 3, 5, 1.2, 0)
            mag, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])
            motion_map = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
            motion_map = np.uint8(motion_map)
            flow_output.write(motion_map)

            # Segmenting into panes: left, middle, right (for both depth and motion maps)
            height, width = motion_map.shape
            part_width = width // 3

            # Slicing the depth map into three equal parts
            left_depth = averaged_depth[:, :part_width]
            middle_depth = averaged_depth[:, part_width:2 * part_width]
            right_depth = averaged_depth[:, 2 * part_width:]

            # Thresholds for detecting if a pane is "covered" in the depth map
            depth_coverage_threshold = 150  # Define a depth threshold for coverage
            coverage_percentage_threshold = 80  # If more than 80% of the pane is covered

            # Check for depth coverage in each pane
            left_depth_coverage = np.mean(left_depth > depth_coverage_threshold) * 100
            middle_depth_coverage = np.mean(middle_depth > depth_coverage_threshold) * 100
            right_depth_coverage = np.mean(right_depth > depth_coverage_threshold) * 100

            alert_messages = []
            detected_pane = None  # Variable to store detected pane

            # Check if the left pane is fully covered based on depth map
            if left_depth_coverage > coverage_percentage_threshold:
                alert_messages.append("Left Pane Fully Covered by Obstacle (Depth Map)")
                cv2.rectangle(frame, (0, 0), (part_width, height), (255, 0, 0), 2)  # Blue box for depth
                detected_pane = "Left"

            # Check if the middle pane is fully covered based on depth map
            if middle_depth_coverage > coverage_percentage_threshold:
                alert_messages.append("Middle Pane Fully Covered by Obstacle (Depth Map)")
                cv2.rectangle(frame, (part_width, 0), (2 * part_width, height), (255, 0, 0), 2)  # Blue box for depth
                detected_pane = "Middle"

            # Check if the right pane is fully covered based on depth map
            if right_depth_coverage > coverage_percentage_threshold:
                alert_messages.append("Right Pane Fully Covered by Obstacle (Depth Map)")
                cv2.rectangle(frame, (2 * part_width, 0), (width, height), (255, 0, 0), 2)  # Blue box for depth
                detected_pane = "Right"

            # Segmenting the motion map into left, middle, and right
            left_motion = motion_map[:, :part_width]
            middle_motion = motion_map[:, part_width:2 * part_width]
            right_motion = motion_map[:, 2 * part_width:]

            # Calculating pixel-level motion in each pane
            left_motion_intensity = np.mean(left_motion)
            middle_motion_intensity = np.mean(middle_motion)
            right_motion_intensity = np.mean(right_motion)

            # Dynamic threshold calculation based on standard deviation of motion
            motion_avg = np.mean([left_motion_intensity, middle_motion_intensity, right_motion_intensity])
            motion_std = np.std([left_motion_intensity, middle_motion_intensity, right_motion_intensity])
            dynamic_threshold = motion_avg + motion_std * 0.5  # Adjust factor as needed

            # Classifying panes where significant motion is detected
            if left_motion_intensity > dynamic_threshold:
                alert_messages.append("Obstacle moving in the Left Pane")
                cv2.rectangle(frame, (0, 0), (part_width, height), (0, 0, 255), 2)  # Red box for motion
                detected_pane = "Left"

            if middle_motion_intensity > dynamic_threshold:
                alert_messages.append("Obstacle moving in the Middle Pane")
                cv2.rectangle(frame, (part_width, 0), (2 * part_width, height), (0, 0, 255), 2)  # Red box for motion
                detected_pane = "Middle"

            if right_motion_intensity > dynamic_threshold:
                alert_messages.append("Obstacle moving in the Right Pane")
                cv2.rectangle(frame, (2 * part_width, 0), (width, height), (0, 0, 255), 2)  # Red box for motion
                detected_pane = "Right"

            # Display the alert messages for panes where obstacles are moving or covered
            for message in alert_messages:
                print(f"Frame {frame_count}: {message}")

            # Store detected pane in dictionary
            if detected_pane:
                frame_obstacle_dict[frame_count] = detected_pane

        # Update the previous frame for optical flow calculation in the next iteration
        prev_output = averaged_depth
        frame_count += 1

    # Save the processed frame to output video
    out.write(frame)

    # Display the depth map and the motion map
    # cv2.imshow('Averaged Depth Map', averaged_depth)
    # cv2.imshow('Motion Map', motion_map)
    # cv2.imshow('CV2Frame', frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()  # Save the output video
depth_out.release()  # Save the depth map video
flow_output.release()
cv2.destroyAllWindows()